# Practical session 3: Parallel behaviors and more sensing abilities

In the last practical session, we saw how to define, attach, start, stop and detach a behavior on an E-Puck robot. We implemented three distinct behaviors: `slow_down`, `fear` and `aggression`.

In this section we will see more sensing abilities the E-Puck is equipped with, will define new behaviors using them and will see how to deal with multiple behaviors running in parallel on the same robot. At the end of the session, we will also see how to attach those behaviors on multiple robots interacting together within a V-REP scene.

Open V-REP and load the scene `epuck-scene-3.ttt` located in the directory `Documents/robotics/pyvrep_epuck/vrep-scenes`.

As usual, start the server, the interface and open the simulator session:

## Comment : 

- Make sure to untick and tick back the hide non existing button in interface

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
start_server_and_interface(scene_name="session_3")

/home/cleger/Desktop/code/vivarium/vivarium/utils

STARTING SERVER


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[2024-12-02 18:42:36,997][__main__][INFO] - Scene running: session_3
[2024-12-02 18:42:39,552][vivarium.simulator.simulator][INFO] - Simulator initialized

STARTING INTERFACE


2024-12-02 18:42:42,390 Starting Bokeh server version 3.3.4 (running on Tornado 6.4)
2024-12-02 18:42:42,390 User authentication hooks NOT provided (default user enabled)
2024-12-02 18:42:42,391 Bokeh app running at: http://localhost:5006/run_interface
2024-12-02 18:42:42,391 Starting Bokeh server with process id: 68947
2024-12-02 18:42:44,121 An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
2024-12-02 18:42:45,812 WebSocket connection opened
2024-12-02 18:42:45,839 ServerConnection created


In [4]:
controller = NotebookController()

Start the simulator:

In [5]:
controller.run()

## Selectively detecting scene objects

To define a repertoire of interesting behaviors, we need the robot to selectively sense the proximity of different types of objects. For example, we might want to define a behavior for obstacle avoidance and another one for attraction towards mates. The first behavior will require the proximity from walls and pillars, whereas the second will require the proximity from other robots (although there is only one robot in the scene for now, we'll add more at the end of this session). 

We can filter the result returned by the E-Puck's proximeters by providing the argument `tracked_objects` to the `prox_activations` function:

In [6]:
agent = controller.agents[0]

In [7]:
agent.print_infos()

Entity Overview:
--------------------
Type: AGENT
Subtype: robots
Idx: 0
Exists: True
Position: x=68.65, y=71.90

Sensors: Left=0.79, Right=0.50
Motors: Left=0.00, Right=0.00



In [8]:
agent.left_motor = 0.5

In [9]:
left, right = agent.sensors()
print(left, right)

0.794947624206543 0.5001545548439026


In [10]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds


Print the existing subtypes in the simulation (robots: blue, obstacles: orange, resources: green):

In [11]:
controller.print_subtypes_list()

['obstacles', 'robots', 'resources']


Try to detect only 1 type (e.g obstacles):

In [12]:
left, right = agent.sensors(sensed_entities=["obstacles"])
print(left, right)

0.8254112005233765 0


Can also do it for multiple types :

In [13]:
left, right = agent.sensors(sensed_entities=["robots", "obstacles"])
print(left, right)

0.8332110643386841 0


Automatically gives an error message if give a non existing type or spell a type wrong :

In [14]:
left, right = agent.sensors(sensed_entities=["non_existing_type"])

FPS: 32.50


AssertionError: Please specify valid sensed entities among {'obstacles', 'robots', 'resources'}

Indeed tells use to pick a type among ['obstacles', 'robots', 'resources']

Executing the cell above will return the proximeter activations only for the `Cup` objects (the kind of trash bins in the V-REP scene). Give it a try by moving a cup in the detection area of the proximiters and re-executing the cell above to observe the change in the returned values. You can also check that the proximeter activations are not modified by other objects such as pillars.

The `tracked_objects` argument requires a list of strings (`["Cup"]` in the example above). In Python, a list is a collection of values separated by commas and surrounded by square bracket: `["Cup"]` is therefore a list of only one element (the string `"Cup"`), whereas `["Cup", "ePuck"]` is a list of two elements (the strings `"Cup"` and `"ePuck"`). 
The `tracked_objects` argument, as its name indicates, sets the objects to be tracked by the proximeters. Each object in a V-REP scene has a name, which is shown when you select an object by clicking on it in the interface. You can also inspect the names of all the objects in the `Scene hierarchy` panel on the left (if not visible, you can activate in the `Tools` menu). For example, we see that the cups have the names `Cup`, `Cup0`, `Cup1` etc ..

In the cell above, `tracked_objects=["Cup"]` means *only return the proximeter activations of objects having their names starting with `"Cup"`*. Since only cups have their names starting with `Cup`, it will return the proximeter activation only for cups, not considering e.g. walls and pillars.

**Q1:** Write the code printing the proximeter activations for walls and pillars and test that it works as expected by placing your epuck close to those objects and verifying the returned values:

In order to track several types of objects at the same time, we can pass several strings to `tracked_objects`. For example, if we want to track both cups and trees, we will write:

In [15]:
left, right = agent.sensors(sensed_entities=["resources", "obstacles"])
print(left, right)

0 0.4422719478607178


This is because all trees in the scene have their name starting with `Tree` (`Tree`, `Tree#0`, `Tree#1` etc..., as shown in the `Scene hierarchy` panel).

**Q2:** Define an `obstacle_avoidance` behavior. Obstacles are walls, pillars and trees, but not cups.  The robot has to turn in the direction opposite to the obstacle, with its speed inversely proportional to the proximeter activations (the closer an obstacle, the lower the speed). 
*Tip:* it is similar to the `shyness` behavior of [Braitenberg vehicles](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0).

In [16]:
def obstacle_avoidance(agent):
    left, right = agent.sensors(sensed_entities=["obstacles"])
    return 1. - right, 1. - left

Remember that to test a behavior, you first have to detach all behaviors that could still be attached to the E-Puck, then to attach and start the new one, that is:

In [17]:
agent.detach_all_behaviors()
agent.attach_behavior(obstacle_avoidance)

## Comment : 

- Might be normal but seems like the agent has already a foraging behavior (because sometimes follows resources)
- Might be because he can't sensed them and when it goes on a straight line, it keeps pushing resources and have them in from of it

In [18]:
agent.detach_all_behaviors(stop_motors=True)

In [19]:
agent.print_infos()
agent.print_behaviors()

Entity Overview:
--------------------
Type: AGENT
Subtype: robots
Idx: 0
Exists: True
Position: x=66.23, y=62.56

Sensors: Left=0.17, Right=0.49
Motors: Left=0.00, Right=0.00

No behaviors attached


## Environmental dynamics

Until now the environment in which the E-Puck is evolving is quite static: although some objects can be pushed by the robot (e.g. the cups), there is nothing that appears or disappears in the environment. We are now going to see how we can generate food sources appearing at random positions in the environment and disappearing whenever a robot eat them. A food source is modeled as a V-REP `Sphere` object, meaning that it can roll on the floor. Making such spheres to appear at regular time intervals and at random positions in the environment is done with:

## Comment 

- to make sure it works well make the robot turn on himself with following cmd 

In [20]:
agent.left_motor = 0.8
agent.right_motor = 0.0

We want to implement a mechanism where agents will eat only ressources. To do so, we have to do three things:

- Add `resources` to the agents's diets
- Start the eating mechanism in the controller
- Start the apparition of `resources` in the environment

### Manipulate the diet of agents

In [21]:
print("Different subtypes of entities:")
controller.print_subtypes_list()

print(f"\nAgent eating information:")
print(f"Current agent diet: {agent.diet}")
print(f"Current agent eating range: {agent.eating_range}")

Different subtypes of entities:
['obstacles', 'robots', 'resources']

Agent eating information:
Current agent diet: []
Current agent eating range: 10


You can also get theses informations by using the print_infos() function, with `full_infos` set to True: 

In [22]:
agent.print_infos(full_infos=True)

Entity Overview:
--------------------
Type: AGENT
Subtype: robots
Idx: 0
Exists: True
Position: x=61.76, y=63.96

Sensors: Left=0.56, Right=-0.00
Motors: Left=0.80, Right=0.00

Diet: []
Eating range: 10

Configuration Details:
  - color: #0000ff
  - diameter: 10.0
  - exists: True
  - friction: 0.10000000149011612
  - idx: 0
  - left_prox: 0.5611939430236816
  - mass_center: 1.0
  - mass_orientation: 0.125
  - max_speed: 10.0
  - orientation: 2.0870513916015625
  - prox_sensed_ent_idx: [27  1]
  - prox_sensed_ent_type: [2 0]
  - proximity_map_dist: [  0.          77.13118744  99.93045807 101.57701111  27.1563549
   7.69326782  28.69501877  98.90074158  75.92100525 103.78182983
  15.54741955 102.93409729  93.65523529 119.60349274  45.18721008
  20.96422005  94.3768692   50.27866364  77.41730499  57.48917007
 111.10734558  90.07124329  48.02243423  80.30754852 101.11505127
 111.52439117  85.38514709  26.32836151  83.20754242]
  - proximity_map_theta: [ 0.         -0.76636839  0.74832582 

In [23]:
agent.diet = ["resources"]

Now, if the eating mechanism is started, the agents will eat the resources. If we wanted them to eat other kind of entities such as obstacles, we would have to add 'obstacles' to the diet of the agents (make sure to respect the good ortograph of the entities with the controller.print_subtypes_list() function).

### Start the eating mechanism

You can start the eating of the controller mechanism with the command in the following cell. This will make the agents eat the resources in their diet and eating range. You also have an option to only eat the resources in your proximeters range ... don't know if it is useful to tell it here.

In [24]:
controller.start_eating_mechanism()

### Start spawning of resources

Can start ressources apparition, will also start eating mechanism of the server where agents can eat resources when close to them. TO do so, define an interval of steps between each apparition of a resource with the 'interval' parameter. To get a good approximation of this interval, use the get fps function to know how many steps per seconds

In [25]:
controller.print_fps()

measuring the FPS (number of steps per second) in the controller during 2 seconds


e.g. if we want a resource to appear every 5 seconds, we will set the interval to 5*fps

In [26]:
seconds = 2
fps = 28
interval = seconds * fps

controller.start_resources_apparition(interval=interval)

In [27]:
agent.detach_all_behaviors()
agent.attach_behavior(obstacle_avoidance)
agent.start_all_behaviors()

where `period` indicates the time interval at which spheres will appear (here every 5 seconds). In order to stop sphere apparition:

In [28]:
controller.stop_resources_apparition()

You can also remove all the entities of a certain type with this command:

In [29]:
controller.remove_entity_type("resources")

Entity 5 already removed
Entity 6 already removed
Entity 7 already removed
Entity 8 already removed
Entity 9 already removed
Entity 11 already removed
Entity 12 already removed
Entity 13 already removed


FPS: 29.50


If the resources apparition function is still attached to the controller, it will keep spawning resources even after removing them with the precedent function. It will print an information message for all the non existing entities of this type we tried to remove (here, already non-existing ressources).

### Custom position of resources spawning

You can also control the position range where the resources will appear with the `position_range` parameter. This parameter is a list of 4 values: ((x_min, x_max), (y_min, y_max)) where x_min and x_max are the minimum and maximum x coordinates of the spawning area, and y_min and y_max are the minimum and maximum y coordinates of the spawning area. For example, to make resources appear only in the area between x=0 and x=50, and y=100 and y=200:

In [30]:
# first remove all the resources and stop the resources apparition
controller.remove_entity_type("resources")
controller.stop_resources_apparition()
# then, start the spawning apparition in a specific area
controller.start_resources_apparition(interval=interval, position_range=((0, 50), (100, 200)))

Entity 2 already removed
Entity 3 already removed
Entity 4 already removed
Entity 5 already removed
Entity 6 already removed
Entity 7 already removed
Entity 8 already removed
Entity 9 already removed
Entity 10 already removed
Entity 11 already removed
Entity 12 already removed
Entity 13 already removed
Resources apparition is already stopped


You can play with this process a bit, and then stop it.

In [31]:
controller.stop_resources_apparition()

## TODO : Update text

The cell above will generate spheres at random 3D positions $(x, y, z)$ in the scene with $x\in[-2, 2]$, $y\in[-1, 0.5]$ and $z\in[1, 2]$ (analyze the cell above to understand how the `min_pos` and `max_pos` arguments are converted in $(x, y, z)$ intervals and ask us if it is not clear). You can check how the $x, y, z$ axes are oriented at the bottom-right corner of the V-REP scene (try to rotate the scene while looking at it). When selecting an object, you can check the coordinate of its center in the text located in the top-left corner of the scene (`Last selected object position`). The center of the scene is at $x=0, y=0, z=0$. The floor is contained in (approximately) $x\in[-2.5, 2.5]$ and $y\in[-2.5, 2.5]$, with $z=0$ on the surface. 

**Q3:** Write the code that makes sphere appearing just above one of the trees and so that they then roll in various directions:

Whenever the E-Puck is touching a sphere it will "eat" it, meaning that the sphere will disappear from the environment. You can see it by manually moving the E-Puck close to a sphere (the sphere should disappear, although this might take some time). Note however that the spheres will disappear only if the sphere apparition is activated (i.e. if you haven't executed `simulator.stop_sphere_apparition()` as the last command: in that case, you will have to re-execute `simulator.start_sphere_apparition(period=5.)` as above).

If no E-Puck is eating the spheres, you might end up with a large number of spheres occupying the environment and this could dramatically impair the V-REP performances. In that case, clean the environment by closing and restarting the session as explained at the beginning of the notebook. A good practice to avoid the proliferation of spheres is to increase the period (e.g.`simulator.start_sphere_apparition(period=20.)`) and to stop sphere apparition whenever you don't need it.

## Combining behaviors

**Q4:** Define a behavior allowing the robot to catch food sources, let's call it `foraging`. The robot has to orient itself toward food sources, with a speed proportional to the proximiter activations (the closer the food source, the higher the speed) *Tip 1:* It's similar to the `aggression` behavior. *Tip 2:* Generated spheres have their names starting by "Sphere" (you can see it in V-REP in the scene hierarchy panel).

In [32]:
def foraging(agent):
    left, right = agent.sensors(sensed_entities=["resources"])
    return right, left

Attach and start the behavior on the E-Puck: 

In [33]:
# First detach previous behaviors that might still be attached to the robot
agent.detach_all_behaviors()

# Write the code to attach and start the `foraging` behavior below
agent.attach_behavior(foraging)
agent.start_all_behaviors()

In [34]:
agent.print_behaviors()

Available behaviors: ['foraging'], Active behaviors: ['foraging']


Whenever a sphere is detected by the proximeters, the robot should go towards it. However, if at one point the proximeters don't detect any sphere, the robot will probably stop (depending on how you have defined the behavior). The only event that could make the robot move again would be a sphere that rolls within the proximeter detection area, which is not very likely to happen (and therefore quite a bad option if the survival of the robot depends on its foraging abilities). A solution to avoid such a blocking situation is to combine the `foraging` behavior with another one that keeps the robot in movement, as it is for example the case of the `obstacle_avoidance` behavior we have defined before. Let's attach and start the `obstacle_avoidance` behavior, but this time without detaching the previously attached `foraging` behavior:

In [35]:
agent.attach_behavior(obstacle_avoidance)
agent.start_behavior(obstacle_avoidance)

Since we haven't detached the previous behavior, the robot is now executing two behaviors in parallel. This can be checked with:

In [36]:
agent.print_behaviors()

Available behaviors: ['foraging', 'obstacle_avoidance'], Active behaviors: ['foraging', 'obstacle_avoidance']


which tells us that both behaviors are attached and started. In V-REP, you can see that the robot is now both foraging and avoiding obstacles. For doing so, the motor activation sent to each wheel corresponds to the average of the motor activation returned by each behavior (this averaging is implemented internally, you don't need to worry about it).

## Dealing with multiple robots

This section explains how to deal with multiple robots and how to attach different behaviors to them.

Then restart the notebook (`Kernel -> Restart`).

Select the E-Puck robot by clicking on it in the scene. Copy-paste it, either using the `Edit` menu, or by pressing the usual editing shortcut `Ctrl-C` then `Ctrl-V`. A new E-puck will be placed in the scene at the exact same position as the previous one. Drag and drop this new robot to another position. Now you should see two robots in the scene.

Re-open a session, this time requesting the references to two E-Pucks instead of one, by executing:

### comment 

- can remove all objects with line below

In [37]:
# controller.remove_objects(object_type=resources_id)

In [38]:
# Make all agents spawn with the exists flag
for agent in controller.agents:
    agent.exists = True
    agent.diet = ["resources"]

agent_0 = controller.agents[0]
agent_1 = controller.agents[1]

Now you have access to the two E-Pucks through the variables `epuck1` and `epuck2` (these variables names are arbitrary, you can choose whatever you want, e.g. `predator` and `prey`). You can attach and start behaviors on each E-Puck independently, in the same way as you did before, simply using either the `epuck1` and `epuck2` variables instead of only `epuck` one as before.

As an example, let's say we want to attach the `obstacle_avoidance` behavior we have defined above to `epuck1`, and both the `obstacle_avoidance` and the `foraging` behaviors to `epuck2`. 
Since we have restarted the notebook, you first need to re-execute the cells defining both behaviors above (i.e. the cells starting with `def obstacle_avoidance(epuck):` and `def foraging(epuck):`).

In [39]:
agent_1.attach_behavior(obstacle_avoidance)
agent_1.attach_behavior(foraging)
agent_1.start_all_behaviors()

In [40]:
controller.stop_resources_apparition()
controller.start_resources_apparition(interval=interval)
controller.start_eating_mechanism()

Resources apparition is already stopped


The code will be:

--> Old Code I think

In [41]:
# As usual, we detach the possibly already running behaviors.
# Since we now have two epucks, called `epuck1` and `epuck2`, we have to do it on each of them.
agent_0.detach_all_behaviors()
agent_1.detach_all_behaviors()

# Then we attach the obstacle avoidance behavior to epuck1:
agent_0.attach_behavior(obstacle_avoidance)

# Then we attach both the obstacle avoidance and the foraging behavior to epuck2:
agent_1.attach_behavior(obstacle_avoidance)
agent_1.attach_behavior(foraging)

# Finally, we start the attached behaviors on each epuck
agent_0.start_all_behaviors()  # This will start obstacle_avoidance on epuck1 (because it is the only behavior we have attached to epuck1)
agent_1.start_all_behaviors()  # This will start both obstacle_avoidance and foraging on epuck2 (because we have attached both behaviors on epuck2)

**Q5:** Implement the `fear` and `aggression` behaviors so that they are directed only toward the other E-Puck, using the `tracked_objects` argument of the `prox_activations` function as we have seen above. Then attach both the `obstacle_avoidance` and the `aggression` behaviors to one E-Puck, and both the `obstacle_avoidance` and the `fear` behaviors on the second. If you did it well, you should observe a simple "prey-predator" interaction, where `epuck1` tries to catch `epuck2` and `epuck2` tries to escape from `epuck1`.

In [42]:
for ag in controller.agents:
    ag.detach_all_behaviors(stop_motors=True)

In [43]:
agent_0 = controller.agents[0]
agent_1 = controller.agents[1]

In [44]:
def aggression(agent):
    left, right = agent.sensors(sensed_entities=["robots"])
    return right, left

def fear(agent):
    left, right = agent.sensors(sensed_entities=["robots"])
    return left, right

In [45]:
agent_0.attach_behavior(aggression)
agent_0.attach_behavior(obstacle_avoidance)
agent_0.start_all_behaviors()
agent_0.diameter = 10.
agent_0.color = 'red'
agent_0.wheel_diameter = 2.5

agent_1.attach_behavior(fear)
agent_1.attach_behavior(obstacle_avoidance)
agent_1.start_all_behaviors()
agent_1.diameter = 4.
agent_1.color = 'cyan'
# Make the little agent go faster
agent_1.wheel_diameter = 4.

## Agent sensors

Let the simulation running and observe the behaviors ... you can play with the speed of the agents by changing their wheel diameter ... 

You should observe that the agent0 is chasing the agent1 and the agent1 is avoiding agent0.... But because the sensors of agent1 are only directed in the forward direction, it is really hard to avoid the agent0 when it get behind him because cannot see it ... Additionally, the agent0 is a predator but has a pretty bad vision because it can't see very far ...

We can fix this by modifying the sensors of the agents to change their angle and their range by using the following commands : 

First restart the session to clean the environment. Close it:

In [46]:
agent_0.proxs_dist_max = 100.
agent_0.proxs_cos_min = 0.8

Now change the code to make the sensors of the prey agent to make him react to the predator agent even if it is a little bit behind him and reduce its range ...

You can also change the speed of both agents to make the simulation more interesting (e.g the predator is not just stuck behind the prey because they have the same speed)

In [47]:
# example
agent_1.proxs_dist_max = 40.
agent_1.proxs_cos_min = -0.9

### Comment : 

Could do a question like that (improve it):

What would be the advantages of having a better vision for the predator agent ? For the prey agent ? What would be the optimal parameters for both if you had to choose a trade-off between (e.g because increasing range / angle of sensors would cost more energy for an agent) the max range and the angle of the sensors in your opinion ?

In [48]:
controller.stop()
stop_server_and_interface(safe_mode=True)

 Found the process scripts/run_interface.py running with this PID: 68947
 Found the process scripts/run_server.py running with this PID: 68709


Simulator is already stopped



Stopping server and interface processes



Killed process with PID: 68947
Killed process with PID: 68709

Server and Interface processes have been stopped



Received signal 15, shutting down


False

Now that you finished session 3, you can now jump to the notebook of [session 4](session_4.ipynb).